In [1]:
import sys
sys.path.insert(0, './../../../Models')
from sphere_points import generate_points

import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt

import torch
torch.manual_seed(0)
import torch.nn as nn
from tqdm import tqdm
from torch.optim import SGD
from torch.nn.functional import normalize
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

In [2]:
# device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
# device = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
def plot_losses(losses):
    losses = np.array(losses)
    n_dims, epochs = losses.shape
    plt.figure(figsize = (12, 5))
    for l in range(n_dims):
        plt.subplot(1, n_dims, l + 1)
        plt.plot(1 + np.arange(epochs), losses[l])
        plt.title(f"Layer {l + 1} Loss")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
    plt.tight_layout()
    plt.show()

In [4]:
theta = 1
'''
cos_similarity = nn.CosineSimilarity(dim = 1, eps = 1e-6)
'''
mse_loss = nn.MSELoss()

'''
def loss_layer(vec1, vec2):
    return torch.mean(torch.log(2 - (theta * cos_similarity(vec1, vec2))), dim = 0)
'''
# def loss_layer(vec1, vec2):
    # return torch.mean((theta * (torch.sum(torch.pow(torch.subtract(vec1, vec2), 2), dim=1))), dim = 0)

def loss_layer(vec1, vec2):
    vec1 = normalize(vec1, p = 2, dim = 1)
    vec2 = normalize(vec2, p = 2, dim = 1)
    return torch.mean(torch.log(1 + (theta * torch.sqrt(torch.sum(torch.pow(torch.subtract(vec1, vec2), 2), dim=1)))), dim = 0)

initial = None
num_classes = 10

# Data dimension
# (num_data, num_features) => no dimension for batch size please
class Layer(nn.Linear):
    def __init__(self, in_features, out_features, bias, device, lr, apply_dropout=False):
        super().__init__(in_features, out_features, bias, device)
        self.out_features = out_features
        self.bias_flag = bias
        self.lr = lr
        self.num_classes = num_classes
        self.dimension = out_features
        # self.leaky_relu = nn.LeakyReLU(negative_slope=0.001)
        # self.opt = Adam(self.parameters(), lr = self.lr)
        # self.scheduler = torch.optim.lr_scheduler.MultiStepLR(self.opt,
        #                                                       milestones=[60], gamma=0.1)
        # nn.init.kaiming_normal_(self.weight, mode='fan_in')
        fc1_limit = np.sqrt(6.0 / in_features)
        torch.nn.init.uniform_(self.weight, a=-fc1_limit, b=fc1_limit)
        self.dropout = nn.Dropout(0.1)
        self.apply_dropout = apply_dropout
        global initial
        self.directions = generate_points(self.num_classes, self.dimension, steps = 10000)
        initial = np.array(self.directions)
        self.directions = [torch.tensor(t, dtype = torch.float32).to(device) for t in self.directions]

    def train(self, x, label):
        if self.apply_dropout:
            x = self.dropout(x)
        y = self.forward(x) # shape: (num_data, out_features)
        # y = normalize(y, p = 2, dim = 1)
        activation = nn.LeakyReLU(negative_slope=0.001)
        opt = SGD(self.parameters(), lr=self.lr)
        if self.bias_flag:
            y = activation(torch.mm(x, self.weight.T) + self.bias.unsqueeze(0))
        else:
            y = activation(torch.mm(x, self.weight.T))

        directions = torch.zeros_like(y)
        for i in range(y.shape[0]):
            directions[i, :] = self.directions[label[i]]

        loss = loss_layer(y, directions)
        opt.zero_grad()
        loss.backward(retain_graph = True)
        opt.step()
        # self.scheduler.step()

        # normalize the directions
        # self.directions.data = normalize(self.directions.data, p = 2, dim = 1)

        return loss.detach().item(), y

    def test(self, x):
        with torch.no_grad():
            # y = self.forward(x)
            activation = nn.LeakyReLU(negative_slope=0.001)
            if self.bias_flag:
                y = activation(torch.mm(x, self.weight.T) + self.bias.unsqueeze(0))
            else:
                y = activation(torch.mm(x, self.weight.T))
        max_idx_list =[]
        for dat in range(y.shape[0]):
            max = np.inf
            max_idx = 0
            for i in range(self.num_classes):
                # euc_dist = torch.mean(torch.norm(y[dat, :].unsqueeze(0) - self.directions[i].reshape(1, -1), dim=1), dim=0)
                # euc_dist = torch.sum(torch.pow(torch.subtract(y[dat, :].unsqueeze(0), self.directions[i].reshape(1, -1)), 2), dim=1)
                euc_dist = torch.sqrt(torch.sum(torch.pow(torch.subtract(y[dat, :], self.directions[i]), 2)))
                if euc_dist < max:
                    max = euc_dist
                    max_idx = i
                '''
                cos_sim = cos_similarity(y[dat, :].unsqueeze(0), self.directions[i].reshape(1, -1))
                if cos_sim > max:
                    max = cos_sim
                    max_idx = i
                '''
            max_idx_list.append(max_idx)
        return torch.tensor(max_idx_list, device=device), y

In [5]:
class Net(nn.Module):
    def __init__(self, dims_list, bias, epochs, lr, device):
        super(Net, self).__init__()
        self.dims_list = dims_list
        self.bias = bias
        self.epochs = epochs
        self.lr = lr
        self.device = device
        self.layers = []
        # self.sigmoid = nn.Sigmoid()
        global initial
        for d in range(len(self.dims_list) - 1):
            print(f"Initialization {d + 1} / {len(self.dims_list) - 1}")
            self.layers += [Layer(self.dims_list[d], self.dims_list[d + 1], self.bias, self.device, self.lr)]
            print("Complete\n")

    def train(self, train_loader, test_loader):
        layer_loss_list = []
        acc_train = []
        acc_test = []
        layer_w = [[] for _ in range(len(self.dims_list) - 1)]

        for i in range(len(self.layers)):
            layer_loss_list.append([])
        pbar = tqdm(total = self.epochs * len(train_loader) * len(self.layers),
                    desc = f"Training", position = 0, leave = True)
        '''
        # Test the network
        with torch.no_grad():

            for i in range(len(self.layers)):
                layer_w[i].append(torch.norm(self.layers[i].weight, p=2).item())
            acc_train.append(net.test(trainloader))
            acc_test.append(net.test(testloader))
        '''

        for epoch in range(self.epochs):

            if epoch and not (epoch % 10): 
                # learning rate decay
                for i in range(len(self.layers)):
                    self.layers[i].lr = self.layers[i].lr - 0.1
                    print('lr decreased to ', self.layers[i].lr)

            loss_agg = [0] * len(self.layers)
            for dat in train_loader:
                x, label = dat
                x = x.to(device)
                label = label.to(device)
                for i in range(len(self.layers)):
                    loss, y = self.layers[i].train(x, label)
                    self.layers[i].zero_grad(set_to_none=True)
                    x = y.detach()
                    loss_agg[i] += loss / len(train_loader)
                    del y
                    pbar.update(1)
            pbar.set_postfix(epoch = epoch + 1, loss = loss_agg)
            for i in range(len(self.layers)):
                layer_loss_list[i].append(loss_agg[i])

        # Test the network
        with torch.no_grad():
            for i in range(len(self.layers)):
                layer_w[i].append(torch.norm(self.layers[i].weight, p=2).item())
            acc_train.append(net.test(trainloader))
            acc_test.append(net.test(testloader))

        pbar.close()
        return [layer_loss_list, acc_train, acc_test, layer_w]

    def test(self, data_loader):
        all_accuracy = []
        correct = [0 for _ in range(len(self.layers))]
        total = [0 for _ in range(len(self.layers))]
        for dat in data_loader:
            x = dat[0]
            label = dat[1]
            x = x.to(device)
            label = label.to(device)
            num = label
            preds = []
            for i in range(len(self.layers)):
                pred, x = self.layers[i].test(x)
                preds.append(pred)

            for i in range(len(preds)):
                correct[i] += (preds[i] == num).sum().item()
                total[i] += num.shape[0]

        all_accuracy.append(np.array(correct) / total[-1])
        return all_accuracy

In [6]:
flatten_transform = transforms.Lambda(lambda x: x.view(x.size(0), -1).squeeze())

# # Define data transformations
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.,), (0.5,)),
#     flatten_transform
# ])
transform = transforms.Compose([transforms.ToTensor(), flatten_transform]) # this normalizes to [0,1]
trainset = torchvision.datasets.MNIST(root='./../../Data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./../../Data', train=False, download=True, transform=transform)
batch_size = 50
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)


# full_data_set = torch.utils.data.ConcatDataset([trainset, testset])

In [7]:
# ratio = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
ratio = [0.2]
test_acc = []
train_acc = []
w_norm = []

num_runs = 5

# for r in ratio:
    # print(f"train ratio: {r}")
'''
test_size = int(r * len(full_data_set))
train_size = len(full_data_set) - test_size
train_dataset, test_dataset = torch.utils.data.random_split(full_data_set, [train_size, test_size])

# Create DataLoader for training set
trainloader = DataLoader(train_dataset, batch_size = 64, shuffle=True)

# Create DataLoader for test set
testloader = DataLoader(test_dataset, batch_size = 64, shuffle=False)
'''
for _ in range(num_runs):

    dims_list = [784, 1024, 10]
    bias = True # False
    epochs = 200
    lr = 2.5
    num_classes = 10
    net = Net(dims_list, bias, epochs, lr, device)

    # Train the network
    layer_loss_list = net.train(trainloader, testloader)
    '''
    plot_losses(layer_loss_list[0])
    '''

    '''
    # Test the network
    acc_train = net.test(trainloader)
    print(f"Train accuracy: {acc_train * 100:.2f}%")

    acc_test = net.test(testloader)
    print(f"Test accuracy: {acc_test * 100:.2f}%")
    '''

    train_acc.append(layer_loss_list[1])
    test_acc.append(layer_loss_list[2])
    w_norm.append(layer_loss_list[3])

Initialization 1 / 2
Complete

Initialization 2 / 2
Complete



Training:   5%| | 24012/480000 [03:27<1:06:31, 114.24it/s, epoch=10, loss=[0.6553428988158702, 0.592

lr decreased to  2.4
lr decreased to  2.4


Training:  10%| | 48017/480000 [06:55<1:02:33, 115.08it/s, epoch=20, loss=[0.6493785619735714, 0.585

lr decreased to  2.3
lr decreased to  2.3


...

Training:  90%|█████████████████████████████████████████████████████████████████████████████████▉         | 432120/480000 [11:17<01:17, 620.25it/s, epoch=180, loss=[0.6425016787151491, 0.574206484307845]]

lr decreased to  0.6999999999999988
lr decreased to  0.6999999999999988


Training:  95%|██████████████████████████████████████████████████████████████████████████████████████▍    | 456117/480000 [11:54<00:38, 622.13it/s, epoch=190, loss=[0.6424186098575598, 0.574779720455408]]

lr decreased to  0.5999999999999989
lr decreased to  0.5999999999999989


Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 480000/480000 [12:59<00:00, 615.72it/s, epoch=200, loss=[0.6423404491941133, 0.5741494803130628]]


In [8]:
print(train_acc, test_acc)

[[[array([0.90558333, 0.83845   ])]], [[array([0.90391667, 0.92308333])]], [[array([0.905     , 0.90326667])]], [[array([0.9049    , 0.93716667])]], [[array([0.90415   , 0.95371667])]]] [[[array([0.9035, 0.8341])]], [[array([0.9007, 0.9078])]], [[array([0.9035, 0.8937])]], [[array([0.9025, 0.9253])]], [[array([0.9019, 0.9394])]]]


In [9]:
np.save("./../new_data/ablation_normeuc_mnist_train_acc.npy", np.array(train_acc))
np.save("./../new_data/ablation_normeuc_mnist_test_acc.npy", np.array(test_acc))
np.save("./../new_data/ablation_normeuc_w_mnist_end.npy", np.array(w_norm))